# Time Series Prediction with Reservoir Computing ($RCN$)

AUTHOR: Anurag Dutta (anuragdutta.research@gmail.com)

SUPERVISOR: Tanujit Chakraborty (tanujit.chakraborty@sorbonne.ae)

CREATED: 2023-03-07 20:00:49

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Gathering Dependencies

_Importing Required Libraries_

In [2]:
pip install hampel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import LSTM
import keras
import tensorflow as tf
from hampel import hampel
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Loading Datasets

_TURKEY EARTHQUAKE_

In [30]:
data = pd.read_csv("/content/drive/MyDrive/new/datasets/eq.csv")
training_set = data.iloc[:, 4]
print(training_set)

0       4.4
1       5.2
2       4.8
3       4.4
4       5.7
       ... 
6569    4.5
6570    4.0
6571    4.4
6572    4.1
6573    4.0
Name: Magnitude, Length: 6574, dtype: float64


## Preprocessing the Data

In [31]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(training_set, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

Streaming output truncated to the last 5000 lines.
[4.5 4.6 4.2] 4.8
[4.6 4.2 4.8] 5.5
[4.2 4.8 5.5] 4.1
[4.8 5.5 4.1] 4.5
[5.5 4.1 4.5] 7.2
[4.1 4.5 7.2] 5.1
[4.5 7.2 5.1] 5.3
[7.2 5.1 5.3] 4.7
[5.1 5.3 4.7] 4.3
[5.3 4.7 4.3] 4.5
[4.7 4.3 4.5] 4.0
[4.3 4.5 4. ] 4.8
[4.5 4.  4.8] 4.4
[4.  4.8 4.4] 4.2
[4.8 4.4 4.2] 4.1
[4.4 4.2 4.1] 5.2
[4.2 4.1 5.2] 4.4
[4.1 5.2 4.4] 4.3
[5.2 4.4 4.3] 5.1
[4.4 4.3 5.1] 4.1
[4.3 5.1 4.1] 4.0
[5.1 4.1 4. ] 4.3
[4.1 4.  4.3] 4.5
[4.  4.3 4.5] 4.6
[4.3 4.5 4.6] 4.2
[4.5 4.6 4.2] 4.3
[4.6 4.2 4.3] 4.2
[4.2 4.3 4.2] 4.4
[4.3 4.2 4.4] 4.2
[4.2 4.4 4.2] 4.4
[4.4 4.2 4.4] 5.1
[4.2 4.4 5.1] 4.3
[4.4 5.1 4.3] 4.3
[5.1 4.3 4.3] 4.2
[4.3 4.3 4.2] 4.3
[4.3 4.2 4.3] 4.5
[4.2 4.3 4.5] 4.5
[4.3 4.5 4.5] 4.7
[4.5 4.5 4.7] 4.4
[4.5 4.7 4.4] 4.5
[4.7 4.4 4.5] 4.1
[4.4 4.5 4.1] 4.3
[4.5 4.1 4.3] 4.3
[4.1 4.3 4.3] 4.5
[4.3 4.3 4.5] 4.6
[4.3 4.5 4.6] 5.0
[4.5 4.6 5. ] 4.2
[4.6 5.  4.2] 4.7
[5.  4.2 4.7] 4.3
[4.2 4.7 4.3] 4.2
[4.7 4.3 4.2] 4.3
[4.3 4.2 4.3] 4.7
[4.2 4.3 4.7]

## Model ($RCN$)

In [34]:
pip install reservoirpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
from reservoirpy.nodes import Reservoir, Ridge

reservoir = Reservoir(units=100, lr=0.5, sr=0.001)
readout = Ridge(output_dim=3, ridge=1e-8)

In [36]:
esn = reservoir >> readout

In [37]:
esn.fit(X[:5250], X[1:5251], warmup=100)

Running Model-3:   0%|          | 0/1 [00:00<?, ?it/s]
Running Model-3: 196it [00:00, 1958.24it/s]           
Running Model-3: 392it [00:00, 1883.76it/s]
Running Model-3: 707it [00:00, 2447.75it/s]
Running Model-3: 1005it [00:00, 2654.23it/s]
Running Model-3: 1312it [00:00, 2802.59it/s]
Running Model-3: 1594it [00:00, 2730.41it/s]
Running Model-3: 1929it [00:00, 2927.04it/s]
Running Model-3: 2234it [00:00, 2964.81it/s]
Running Model-3: 2598it [00:00, 3171.40it/s]
Running Model-3: 2965it [00:01, 3322.54it/s]
Running Model-3: 3298it [00:01, 3141.34it/s]
Running Model-3: 3650it [00:01, 3249.82it/s]
Running Model-3: 4003it [00:01, 3329.86it/s]
Running Model-3: 4338it [00:01, 3309.90it/s]
Running Model-3: 4671it [00:01, 3016.36it/s]
Running Model-3: 5250it [00:01, 2978.57it/s]
Running Model-3: 100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


Fitting node Ridge-5...


'Model-3': Model('Reservoir-5', 'Ridge-5')

In [39]:
predictions = esn.run(X[5251:-1])

Running Model-3: 1319it [00:00, 1709.79it/s]


## Metrics ($RCN$)

In [40]:
import reservoirpy.observables
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(X[5251:-1], predictions))
mae = np.sqrt(mean_absolute_error(X[5251:-1], predictions))
print('Test RMSE: %.3f' % rmse)
print('Test MAE: %.3f' % mae)

Test RMSE: 0.430
Test MAE: 0.550
